In [1]:
%cd ..

/home/jairo/Documentos/GitHub/kink-antikink-parellel


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import multiprocessing as mp
from functools import partial
from time import time
import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
ch.setFormatter(logging.Formatter('[%(asctime)s] %(message)s', datefmt='%d/%m/%Y - %H:%M:%S'))
logger.addHandler(ch)

In [3]:
from modules.numerical import *

In [4]:
from pathlib import Path

class Config:
    L = 50
    N = int((5/4)*1000)
    dx = 2*L/N
    dt = 0.7*dx

    x_lattice = Lattice(x=(-L, L, dx))
    cm_index = x_lattice.at(x=0)
    collider = KinkCollider(
        x_lattice = x_lattice,
        dt = dt
    )

    v_min = 0.05
    v_max = 0.5
    v_lims = (v_min, v_max)

    lamb_min = 0.075
    lamb_max = 50
    lamb_lims = (lamb_min, lamb_max)

    separation_by_delta = 10

    save_dir = Path('data/new_dataset')
    info_path = save_dir/'info.csv'

Config.N

1250

In [5]:
def get_info():
    if Config.info_path.exists():
        info = pd.read_csv(Config.info_path).to_dict('list')
    else:
        info = {'lamb': [], 'exec_time': []}
    return info

def it_was_not_calculated(v, lamb):
    return not len(tuple(Config.save_dir.glob(f'{v}-{lamb}-*.csv'))) > 0

def init_collisions_queue(vs, lambs):
    collisions_queue = mp.Queue()

    info = get_info()
    for lamb in lambs:
        if lamb in info['lamb']:
            vs = tuple(filter(partial(it_was_not_calculated, lamb=lamb), vs))
        if len(vs):
            collisions_queue.put({
                'vs': vs,
                'lamb': lamb
            })

    return collisions_queue

def collider_task(collider, collisions_queue, output_queue):
    while not collisions_queue.empty():
        point = collisions_queue.get()
        logger.debug('(%s) ~~ Iniciando colisões para lamb=%s'%(mp.current_process().name, point['lamb']))
        local_t0 = time()
        delta = np.sqrt(2/point['lamb'])
        x0 = Config.separation_by_delta*delta/2

        collisions = []
        for v in point['vs']:
            _t0 = time()
            y = collider.collide(
                x0s= (-x0, x0),
                vs= (v, -v),
                lamb= point['lamb'],
                t_final= 2*x0/v + Config.L
            )
            _tf = time()
            collisions.append({
                'v': v,
                'y': y,
                'delay': _tf - _t0
            })

        local_tf = time()
        output_queue.put({
            'lamb': point['lamb'],
            'exec_time': local_tf - local_t0,
            'collisions': collisions
        })

def manager_task(collisions_queue, output_queue):
    info = get_info()
    total = collisions_queue.qsize()
    received = 0
    while received < total:
        output = output_queue.get()
        received += 1
        logger.debug('({}) Salvando os resultados para lambda={} ({}%)'.format(
            mp.current_process().name, 
            output['lamb'], 
            received/total * 100
        ))

        for collision in output['collisions']:
            pd.DataFrame(collision['y']).to_csv(
                Config.save_dir/'{v}-{lamb}-{delay}.csv'.format(
                    lamb= output['lamb'], 
                    v= collision['v'], 
                    delay= collision['delay']
                ),
                index= False,
                header= False
            )
        
        if output['lamb'] in info['lamb']:
            info['exec_time'][info['lamb'].index(output['lamb'])] = output['exec_time']
        else:
            info['lamb'].append(output['lamb'])
            info['exec_time'].append(output['exec_time'])
        pd.DataFrame(info).to_csv(Config.info_path, index=False)

def init_session(n_processes, collisions_queue):
    output_queue = mp.Queue()

    logger.debug(f'({mp.current_process().name}) Iniciando {n_processes} + 1 processos...')
    
    manager = mp.Process(target=manager_task, args=(collisions_queue, output_queue))
    manager.start()

    processes = []
    for _ in range(n_processes):
        process = mp.Process(target=collider_task, args=(Config.collider, collisions_queue, output_queue))
        process.start()
        processes.append(process)
    
    for process in processes:
        process.join()
    manager.join()

In [6]:
P = mp.cpu_count() - 1
P

7

In [7]:
V = np.linspace(*Config.v_lims, P)
V

array([0.05 , 0.125, 0.2  , 0.275, 0.35 , 0.425, 0.5  ])

In [8]:
LAMB = np.linspace(*Config.lamb_lims, P)
LAMB

array([ 0.075     ,  8.39583333, 16.71666667, 25.0375    , 33.35833333,
       41.67916667, 50.        ])

In [10]:
init_session(P, init_collisions_queue(V, LAMB))

[07/01/2024 - 00:48:25] (MainProcess) Iniciando 7 + 1 processos...


In [ ]:
import psutil
import pandas as pd

In [ ]:
for k, v in psutil.sensors_temperatures().items(): print(k); print(pd.DataFrame(v))

nvme
       label  current      high  critical
0  Composite    36.85     82.85     89.85
1   Sensor 1    37.85  65261.85  65261.85
2   Sensor 2    40.85  65261.85  65261.85
iwlwifi_1
  label  current  high critical
0           44.0  None     None
coretemp
          label  current   high  critical
0  Package id 0     51.0  100.0     100.0
1        Core 0     51.0  100.0     100.0
2        Core 1     51.0  100.0     100.0
3        Core 2     51.0  100.0     100.0
4        Core 3     51.0  100.0     100.0


In [18]:
36*37/2

666.0

Informações necessárias:

Dado um algoritmo que construa o mosaico de forma gradual, aumentando a resolução para cada 

- Instante em que foi iniciado determinada sessão
- Instante em que cada colisão foi iniciada
- Instante em que cada colisão foi encerrada
-   